# 예제: Deepseek Coder 활용

### 모델 다운

In [ ]:
!ollama pull deepseek-coder-v2:16b

# Deepseek-coder-v2로 게임 Python 코드 생성&디버그

### 라이브러리 호출

In [ ]:
import ollama
import requests
import json
from IPython.display import Markdown, display
!pip install pygame

### 실행 함수

* ollama.chat() 은 ollama 라이브러리의 대화형 API 함수: LLM 모델에 대화 형식(message list) 을 입력하면, 모델이 답변을 생성해서 반환함
* 단순 텍스트 완성(generate)이 아니라 대화 맥락(context) 을 주고받기 위해 chat 함수를 사용
* Local LLM 서버에 채팅 요청을 보냄, 출력은 딕셔너리 형태의 응답(response) 객체
* 대화 히스토리를 messages 리스트로 전달
* 발화자 역할(role)을 사용자로 지정
* 모델이 돌려준 응답(response) 중에서, prompt를 포함한 메시지 본문('content')만 뽑아서 리턴함


In [ ]:
def chat_with_deepseek_coder(prompt, model="deepseek-coder-v2:16b"):
    """Chat with DeepSeek-Coder model via Ollama"""
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {
                    'role': 'user',
                    'content': prompt
                }
            ]
        )
        return response['message']['content']
    except Exception as e:
        return f"Error: {e}. Make sure to run 'ollama pull {model}' first."

### 코드 생성 프롬프트 정의

* ‘블록 부수기’ 게임을 구현하는 소스 코드를 생성하기 위한 프롬프트
* Python 게임 개발자라는 role 부여
* 게임에 포함되어야 하는 기능과 코드를 작성하는 방식을 명시적으로 지시

In [ ]:
generator_prompt = """ 
You are an expert Python game developer specializing in recreating classic arcade games. 
Write complete Python code for a "Brick Breaker" game using the pygame library. 
The game should include: 
   1. A controllable paddle at the bottom of the screen, moved left/right with arrow keys. 
   2. A bouncing ball that can destroy bricks arranged at the top of the screen. 
   3. Bricks arranged in multiple rows with different colors. 
   4. Score tracking and display in the corner. 
   5. Game over when the ball falls below the paddle. 
   6. Victory message when all bricks are destroyed. 
   7. Smooth paddle and ball movement. 
   8. Comments explaining each major part of the code. 
Make sure the code: 
   - Runs without errors. 
   - Uses a game loop with event handling, collision detection, and screen updates. 
   - Has reasonable default settings (window size, paddle speed, ball speed). 
   - Is well-structured and easy to modify. 

Output only the complete Python code inside one code block. 
"""

### 코드 디버그 및 최적화 프롬프트 정의

* Code를 입력으로 받아서 Debug 및 Optimize를 하기 위한 Prompt를 정의
* 출력문에 포함되어야 하는 요소들을 명시적으로 지시


In [ ]:
debug_prompt = """
You are a senior Python developer and performance optimizer.
Debug and optimize the following Python code:

```python
{code}
Please provide:

Corrected & optimized full code (single code block, ready to run)

Explanation of what was wrong (bullet points)

Optimization rationale (algorithmic + implementation-level)

Complexity analysis (before/after: Big-O for time and memory)

Simple benchmark example to compare performance

Key edge-case tests
"""

### 코드 생성

* 생성된 코드를 빈셀에 복사&붙여넣기하여 실행

In [ ]:
response = chat_with_deepseek_coder(generator_prompt)
display(Markdown(response))

In [ ]:
# 여기에 복사 및 붙여넣기



### 코드 디버그 및 최적화

In [ ]:
new_code = chat_with_deepseek_coder(debug_prompt.format(code=response))
display(Markdown(new_code))

* 생성된 코드를 빈셀에 복사&붙여넣기 해서 실행

# Deepseek-coder-v2로 웹 사이트 HTML 코드 생성&디버그

### 라이브러리 호출

In [ ]:
import ollama
import requests
import json
import re
from IPython.display import Markdown, display

### 실행 함수

In [ ]:
def chat_with_deepseek_coder(prompt, model="deepseek-coder-v2:16b"):
    """Chat with DeepSeek-Coder model via Ollama"""
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {
                    'role': 'user',
                    'content': prompt
                }
            ]
        )
        return response['message']['content']
    except Exception as e:
        return f"Error: {e}. Make sure to run 'ollama pull {model}' first."

### 코드 생성 프롬프트 정의

* 웹페이지에 포함되어야 하는 요소들을 명시적으로 지시

In [ ]:
# Code generator
generator_prompt = """
You are a frontend engineer who builds modern, responsive websites.
Write a COMPLETE single-file website (one HTML file) for a product landing page.

The site should include:

1) Header & Navigation
   - Logo (text or simple inline SVG)
   - Navigation links with smooth scroll to sections
   - Mobile-friendly hamburger menu (toggle open/close with JS)
   - Sticky header on scroll

2) Hero Section
   - Clear headline, subheadline, and two CTAs (primary/secondary)
   - Decorative illustration (inline SVG or placeholder)
   - Background gradient or subtle pattern

3) Features Section
   - Grid layout (≥ 6 feature cards) with icons (inline SVG ok)
   - Short titles + descriptions

4) Testimonials
   - At least 3 testimonials with name, role, and quote
   - Simple carousel/slider OR horizontally scrollable cards (no external libs)

5) Pricing
   - 3 tiers (Basic / Pro / Enterprise)
   - One highlighted “Most Popular” plan
   - Feature list bullets and a CTA button per tier

6) FAQ
   - Accordion UI (expand/collapse)
   - At least 6 Q&A items

7) Contact
   - Contact form with name, email, message
   - Client-side validation (required fields, basic email pattern)
   - Success/error message handling (no backend)

8) Footer
   - Copyright, nav links, and social icons (inline SVG)
   - “Back to top” button

Enhancements & Quality Bar:
- Responsive: looks great on mobile, tablet, desktop (use CSS Grid/Flex)
- Accessibility: semantic HTML5, aria-* where helpful, focus states, alt text
- Theming: light/dark theme toggle (CSS variables + localStorage)
- Motion: tasteful transitions/animations (reduced motion respected via prefers-reduced-motion)
- Performance: no external JS/CSS frameworks or CDNs; inline everything
- Code comments: explain each major part (HTML/CSS/JS)
- Clean structure: logical sections, descriptive class names, easy to modify

Technical Requirements:
- Pure HTML + CSS + vanilla JS in ONE file
- Include a minimal CSS reset
- Use CSS variables for colors, spacing, and typography
- Implement smooth scrolling and sticky header behavior
- Implement mobile nav toggle, FAQ accordion, pricing highlight, form validation, theme toggle, back-to-top
- Provide placeholder images/illustrations via inline SVG or data URLs
- All text can be in Korean; write natural marketing copy

Output format:
- Output ONLY the complete HTML (single file) inside one code block.
- Do not include any explanation outside the code block.

"""


### 코드 디버그 및 최적화 프롬프트 정의

In [ ]:
# Debug prompt
debug_prompt = """
You are an expert frontend engineer and code reviewer.
Debug and optimize the following SINGLE-FILE HTML website code:

{code}

Tasks:
1. Check for any HTML, CSS, or JavaScript syntax errors and FIX them.
2. Ensure all required sections (Header, Hero, Features, Testimonials, Pricing, FAQ, Contact, Footer) are present and functional.
3. Verify the required behaviors work correctly:
   - Sticky header and smooth scrolling
   - Mobile nav toggle (hamburger)
   - FAQ accordion expand/collapse (ARIA attributes)
   - Pricing highlight
   - Contact form validation with success/error messages
   - Back-to-top button
   - Theme toggle (light/dark) with localStorage persistence
4. Test responsiveness on mobile, tablet, and desktop. Fix layout or overflow issues.
5. Confirm accessibility: semantic HTML, aria-* attributes, keyboard navigation, focus states.
6. Optimize code:
   - Remove unused styles/JS
   - Use CSS variables consistently
   - Simplify structure where possible
7. Performance check:
   - Inline assets only (no external requests)
   - Respect prefers-reduced-motion for animations
8. Return the corrected & optimized COMPLETE HTML in one code block.
9. After the code, write a short bullet-point summary of:
   - What was wrong
   - What was fixed
   - Any optimizations made
"""

### 코드 생성 및 디버그

In [ ]:
response = chat_with_deepseek_coder(generator_prompt)
new_code = chat_with_deepseek_coder(debug_prompt.format(code=response))
display(Markdown(new_code))

* LLM 출력문에서 HTML 코드 부분을 슬라이싱해서 myWebSite.html로 저장하는 코드

In [ ]:
pattern = r"```html(.*?)```"   # ```html 과 ``` 사이를 캡처
match = re.search(pattern, new_code, re.S)

if match:
    html_code = match.group(1).strip()
    print(html_code)
    
    # 파일로 저장
    with open("myWebSite.html", "w", encoding="utf-8") as f:
        f.write(html_code)

    print("✅ myWebSite.html 저장 완료")

# 실습 문제: WebSite 예제에서 코드 생성 프롬프트의 Header 부분을 수정하여 Logo Text와 배경색을 변경하기 

In [ ]:
import ollama
import requests
import json
import re
from IPython.display import Markdown, display

In [ ]:
def chat_with_deepseek_coder(prompt, model="deepseek-coder-v2:16b"):
    """Chat with DeepSeek-Coder model via Ollama"""
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {
                    'role': 'user',
                    'content': prompt
                }
            ]
        )
        return response['message']['content']
    except Exception as e:
        return f"Error: {e}. Make sure to run 'ollama pull {model}' first."

In [ ]:
# Code generator
generator_prompt = """
You are a frontend engineer who builds modern, responsive websites.
Write a COMPLETE single-file website (one HTML file) for a product landing page.

The site should include:

1) Header & Navigation
   - Logo set to ~.
   - Navigation links with smooth scroll to sections
   - Mobile-friendly hamburger menu (toggle open/close with JS)
   - Sticky header on scroll
   - Background color set ~. 

2) Hero Section
   - Clear headline, subheadline, and two CTAs (primary/secondary)
   - Decorative illustration (inline SVG or placeholder)
   - Background gradient or subtle pattern

3) Features Section
   - Grid layout (≥ 6 feature cards) with icons (inline SVG ok)
   - Short titles + descriptions

4) Testimonials
   - At least 3 testimonials with name, role, and quote
   - Simple carousel/slider OR horizontally scrollable cards (no external libs)

5) Pricing
   - 3 tiers (Basic / Pro / Enterprise)
   - One highlighted “Most Popular” plan
   - Feature list bullets and a CTA button per tier

6) FAQ
   - Accordion UI (expand/collapse)
   - At least 6 Q&A items

7) Contact
   - Contact form with name, email, message
   - Client-side validation (required fields, basic email pattern)
   - Success/error message handling (no backend)

8) Footer
   - Copyright, nav links, and social icons (inline SVG)
   - “Back to top” button

Enhancements & Quality Bar:
- Responsive: looks great on mobile, tablet, desktop (use CSS Grid/Flex)
- Accessibility: semantic HTML5, aria-* where helpful, focus states, alt text
- Theming: light/dark theme toggle (CSS variables + localStorage)
- Motion: tasteful transitions/animations (reduced motion respected via prefers-reduced-motion)
- Performance: no external JS/CSS frameworks or CDNs; inline everything
- Code comments: explain each major part (HTML/CSS/JS)
- Clean structure: logical sections, descriptive class names, easy to modify

Technical Requirements:
- Pure HTML + CSS + vanilla JS in ONE file
- Include a minimal CSS reset
- Use CSS variables for colors, spacing, and typography
- Implement smooth scrolling and sticky header behavior
- Implement mobile nav toggle, FAQ accordion, pricing highlight, form validation, theme toggle, back-to-top
- Provide placeholder images/illustrations via inline SVG or data URLs
- All text can be in Korean; write natural marketing copy

Output format:
- Output ONLY the complete HTML (single file) inside one code block.
- Do not include any explanation outside the code block.

"""


In [ ]:
new_code = chat_with_deepseek_coder(generator_prompt)
pattern = r"```html(.*?)```"   # ```html 과 ``` 사이를 캡처
match = re.search(pattern, new_code, re.S)

if match:
    html_code = match.group(1).strip()
    print(html_code)
    
    # 파일로 저장
    with open("myWebSite.html", "w", encoding="utf-8") as f:
        f.write(html_code)

    print("✅ myWebSite.html 저장 완료")